In [1]:
#### import libraries ###########################################
%load_ext jupyternotify

import pandas as pd
import os
from nltk.corpus import stopwords
import string
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import adjusted_rand_score
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import operator
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import time
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.cluster import AgglomerativeClustering
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from sklearn.cluster import MiniBatchKMeans
import pickle
from gensim.models import Word2Vec
from xgboost import XGBClassifier
from sklearn.preprocessing import scale
import pandas as pd


<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import codecs
from tqdm import tqdm

def load_dataset(path):
    dataset = pd.DataFrame(columns=['X', 'y1', 'y2'])
    #print('Loading dataset...')
    with codecs.open(path, "r", encoding='utf-8', errors='ignore') as fdata:
        for line in tqdm(fdata.readlines()):
            line_split = line.split()
            formated = ' '.join(line_split[:-2])
            dataset.loc[-1] = [formated, line_split[-2], line_split[-1]]  # adding a row
            dataset.index = dataset.index + 1  # shifting index
            dataset = dataset.sort_index()  # sorting by index
    return dataset

In [3]:
def predictions_to_txt(filename, X_test, y_pred):
    i=0
    results = []
    for x in X_test:
        line = []
        line.append(x)
        if y_pred[i] == 1:
            line.append('OFFENSE')
        else:
            line.append('OTHER')
        line.append('OTHER')
        i += 1
        results.append(line)

    predictions = results
    np.savetxt(filename, predictions, fmt=['%s', '%s', '%s'], delimiter='\t', newline='\n', \
                  header='', footer='', encoding='utf-8')

In [4]:
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re

stopwords_german = set(stopwords.words('german'))

usernamePattern = re.compile('@[A-Za-z0-9_]{1,15}')
urlPattern = re.compile('(https?:\/\/)[\/.:\w(1-9)]*\s?')
lbrPattern = re.compile('|LBR|')
stemmer = SnowballStemmer("german")
tkz = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

def tokenize(text):
    text = re.sub(usernamePattern, "",  text)
    text = re.sub(urlPattern, "",  text)
    text = re.sub(lbrPattern, "",  text)
    output = []
    tokens = tkz.tokenize(text)
    for token in tokens:
        if token not in stopwords_german:
            if len(token) > 1:
                if token[0] == '#':
                    token = token[1:]
                output.append(token)
    return output

In [5]:
def tokenize_tfidf(text):
    text = re.sub(usernamePattern, "",  text)
    text = re.sub(urlPattern, "",  text)
    text = re.sub(lbrPattern, "",  text)
    output = []
    tokens = tkz.tokenize(text)
    for token in tokens:
        if token not in stopwords_german:
            if len(token) > 1:
                if token[0] == '#':
                    token = token[1:]
                output.append(stemmer.stem(token))
    return output

In [6]:
def so_load_data(path):
    with open(path) as f:
        content = f.read().splitlines()
    # you may also want to remove whitespace characters like \n at the end of each line
    return [x.rstrip('\\n') for x in content]

In [7]:
dataset = load_dataset(path = '/home/text_mining_project/text_mining_project_2018/evaluation/germeval2018.training.txt')

100%|██████████| 5009/5009 [00:14<00:00, 339.90it/s]


In [8]:
X_test_dataset = so_load_data(path = '/home/text_mining_project/germeval2018.test.txt')

In [9]:
# dataset_full =  pd.concat([dataset, dataset_own])
dataset_full =  dataset

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


X_train_dataset = dataset_full['X'].values
y_train = dataset_full['y1'].values

X_test_raw = X_test_dataset

In [11]:
def encode_label(y):
    np.ones(len(y))
    choose = lambda l : 1 if l == 'OFFENSE' else 0
    return [choose(l) for l in y]

In [12]:
y_train = encode_label(y_train)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

max_features = None
stopwords_german = set(stopwords.words('german'))
# stopwords_german = set(stopwords.words('english'))
tfidf = TfidfVectorizer(tokenizer=tokenize_tfidf, stop_words=stopwords_german, max_features=max_features, ngram_range=(1,3))
print('Transforming documents...')
X_train_tfidf = tfidf.fit_transform(X_train_dataset)
X_test_tfidf = tfidf.transform(X_test_dataset)
print('Transformation finished!')

Transforming documents...
Transformation finished!


In [14]:
model = Word2Vec.load("word2vec_nostem_stopword.model")

In [15]:
from numpy import zeros

def make_bag_of_centroids(X, word_centroid_map, cluster_size):
    centroids_list = []
    for sentence in X:
        centroids = zeros(cluster_size, dtype="float32")
        for word in sentence:
            if word in word_centroid_map:
                centroids[word_centroid_map[word]] += 1
        centroids_list.append(centroids)
    return centroids_list

as_centroid = lambda s: make_bag_of_centroids(s, word2centroid, kmeans_args['n_clusters'])

In [16]:
kmeans_args = {
    'n_clusters': 1000,
}

clustering = MiniBatchKMeans(**kmeans_args).fit_predict(model.wv.vectors)
pickle.dump(clustering, open('./minibatchkmeans.bin', 'wb'))

/home/koogleschreiber/.local/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning:

init_size=300 should be larger than k=1000. Setting it to 3*k



In [17]:
word2centroid = {k: v for k, v in zip(model.wv.index2word, clustering)}

In [18]:
def merge_vector_lists(x1, x2):
    result = []
    for i in range(len(x1)):
        result.append(x1[i] + x2[i])
    return result

In [19]:
stopwords_german = set(stopwords.words('german'))
# stopwords_german = set(stopwords.words('english'))

def preprocess(X):
    return [tokenize(sentence) for sentence in X]

X_preprocess_train = preprocess(X_train_dataset)
X_preprocess_test = preprocess(X_test_dataset)

X_train_centroid = scale(as_centroid(X_preprocess_train))
X_test_centroid = scale(as_centroid(X_preprocess_test))

from scipy.sparse import csr_matrix, hstack

X_train_centroid_sparse = csr_matrix(X_train_centroid)
X_test_centroid_sparse = csr_matrix(X_test_centroid)


X_train = hstack([X_train_centroid_sparse, X_train_tfidf])
X_test = hstack([X_test_centroid_sparse, X_test_tfidf])


In [20]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(penalty="l1", dual=False).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True)
X_train = model.transform(X_train)
X_test = model.transform(X_test)

In [21]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
# 
# scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
# X_train = scaler.fit_transform(X_train.todense())
# X_test = scaler.transform(X_test.todense())
# 
# ch2 = SelectKBest(chi2, k=2000)
# X_train = ch2.fit_transform(X_train, y_train)
# X_test = ch2.transform(X_test)

In [22]:
import warnings

seed = 42
k = 7
jobs = -1

warnings.filterwarnings("ignore")

In [23]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB(alpha=0.1, binarize=0, class_prior=None, fit_prior=False)
clf.fit(X_train, y_train)
predictions=clf.predict(X_test)

In [24]:
predictions

array([0, 1, 1, ..., 0, 0, 0])

In [25]:
predictions_to_txt('upInf_coarse_2.txt', X_test_raw, predictions)